In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv
import plotly.graph_objects as go


from laser_ion_interpol_tRecX_data import process_all_laser_files

from __init__ import FourierTransform, InverseFourierTransform, Fourier_filter
#from calc_kernnel_johannes import IonProb, IonRate
from field_functions import LaserField
from kernel import IonProb, IonRate, analyticalRate


params={'Multiplier': 24.885006619192506, 'Ip': 0.5, 'αPol': 4.51, 'gamma': 3.0012573529411766, 'e0': 2.000419117647059, 'a0': 1, 'a1': 1.2506286764705883, 'b0': -0.00010482328986949341, 'b1': 1, 'b2': 6.127, 'c2': 0.37339566825214854, 'p1': 3.5, 'd1': 8.156654923237362}



# Multiplier=24.885006619192506
# αPol = 4.51
# Ipot=0.5
# dTi = 1
# p1 = 3.5
# a1 = 1
# b2 = 1
# gamma= 3/(2*Ipot)
# c1=111.927
# b3=0.970771
# e1 = 1 + 1/Ipot/2
# b1 = -0.25 + 0.75/Ipot
# a2 = (-7.5 + 30*Ipot - 37.5*Ipot**2 + 15*Ipot**3) / 7.5



class AU:
    meter = 5.2917720859e-11 # atomic unit of length in meters
    nm = 5.2917721e-2 # atomic unit of length in nanometres
    second = 2.418884328e-17 # atomic unit of time in seconds
    fs = 2.418884328e-2 # atomic unit of time in femtoseconds
    Joule = 4.359743935e-18 # atomic unit of energy in Joules
    eV = 27.21138383 # atomic unit of energy in electronvolts
    Volts_per_meter = 5.142206313e+11 # atomic unit of electric field in V/m
    Volts_per_Angstrom = 51.42206313 # atomic unit of electric field in V/Angström
    speed_of_light = 137.035999 # vacuum speed of light in atomic units
    Coulomb = 1.60217646e-19 # atomic unit of electric charge in Coulombs
    PW_per_cm2_au = 0.02849451308 # PW/cm^2 in atomic units
AtomicUnits=AU

def Vector_potential(t,lam0, I, cep, FWHM, pi=np.pi, N=8): 
    """" return A(t) for given Full with half maximum and phase with a cos8 envelope  """
    w0=2*pi*137.035999/(lam0/5.2917721e-2)
    A0=np.sqrt(I/1e15*0.02849451308/w0**2)
    tau_injection=pi*FWHM/(4*np.arccos(2**(-1/(2*N))))
    t=np.asarray(t)
    # Field=np.zeros(np.shape(t))
    # cond=(abs(t)<=tau_injection)
    # t=t[cond]
    # Field[cond]=ne.evaluate("cos(pi*t/tau_injection/2)**N*sin(i*w0*t-cep)*A0")
    #Field= cos(pi*t/tau_injection/2)**N*sin(w0*t-cep)*A0  
    Field=np.where(np.abs(t)<=tau_injection,np.cos(pi*t/tau_injection/2)**N*np.sin(w0*t-cep)*A0,0)
    return Field

def Electric_Field(t,lam0, I, cep, FWHM, pi=np.pi, N=8): 
    """" return E(t) for given Full with half maximum and phase with a cos8 envelope  """
    #print('Electric_Field is deprecated, use Vector_potential instead')
    w0=2*pi*137.035999/(lam0/5.2917721e-2)
    A0=np.sqrt(I/1e15*0.02849451308/w0**2)
    tau_injection=pi*FWHM/(4*np.arccos(2**(-1/(2*N))))
    t=np.asarray(t)
    Field= np.where(np.abs(t)<=tau_injection,np.cos(pi*t/tau_injection/2)**N*np.cos(w0*t-cep)*w0-(N/2/tau_injection*pi*np.cos(pi*t/tau_injection/2)**(N-1)*np.sin(pi*t/tau_injection/2)*np.sin(w0*t-cep)),0)
    return -1*Field*A0

def analytical_rate(field, c1, e1, a1, a2, b1, b2, b3, p1, dTi, Ip):
    field=field+1e-20
    term1 = 1 / (2**(1/4) * (8 * e1**2 * field**2 + Ip)**(1/4) * abs(field)**(9/2)+1e-20)
    term2 = np.exp((4 * (np.sqrt(2) * np.sqrt(8 * e1**2 * field**2 + Ip) - 4 * e1 * abs(field)) * 
                    (-4 * e1**2 * field**2 - Ip + np.sqrt(2) * e1 * np.sqrt(8 * e1**2 * field**2 + Ip) * abs(field))) / 
                   (3 * abs(field)))
    term3 = field**2 * np.sqrt(np.pi)
    term4 = (dTi - 4 * e1 + (np.sqrt(2) * np.sqrt(8 * e1**2 * field**2 + Ip)) / abs(field))**-p1
    term5 = (a1 * field**4 + 8 * (-b1 * e1 * field**4 + 16 * a2 * e1**2 * field**6 - 256 * b2 * e1**3 * field**6 + 
                                  1024 * b3 * e1**4 * field**6 + a2 * field**4 * Ip - 24 * b2 * e1 * field**4 * Ip + 
                                  128 * b3 * e1**2 * field**4 * Ip + 2 * b3 * field**2 * Ip**2) - 
             16 * np.sqrt(2) * (-b2 + 8 * b3 * e1) * field**2 * (8 * e1**2 * field**2 + Ip)**(3/2) * abs(field) + 
             2 * np.sqrt(2) * (b1 - 16 * e1 * (a2 + 4 * e1 * (-3 * b2 + 8 * b3 * e1)) * field**2) * 
             np.sqrt(8 * e1**2 * field**2 + Ip) * abs(field)**3)
    
    return term1 * c1 * term2 * term3 * term4 * term5

def write_csv_rate(filename, time, field_pump, field_probe, ion_nonAdiabatic, ion_analytical):
    with open(filename, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(['time', 'field_pump', 'field_probe', 'ion_nonAdiabatic', 'ion_analytical'])
        for i in range(len(time)):
            writer.writerow([time[i], field_pump[i], field_probe[i], ion_nonAdiabatic[i], ion_analytical[i]])

def write_csv_prob(filename, delay, ion_y, ion_QS, ion_NA):
    with open(filename, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(['delay', 'ion_y', 'ion_QS', 'ion_NA'])
        for i in range(len(delay)):
            writer.writerow([delay[i], ion_y[i], ion_QS[i], ion_NA[i]])

def process_laser_files(base_dir, input_dir, output_interpol, output_ion_prob, output_extract_field):
    process_all_laser_files(base_dir, input_dir, output_interpol, output_ion_prob, output_extract_field)

def read_ion_Prob_data(file_path):
    data = pd.read_csv(file_path, header=None)
    delay = pd.to_numeric(data.iloc[2].iloc[2:].values)[:-1]
    ion_y = pd.to_numeric(data.iloc[1].iloc[2:].values)[:-1]
    return delay, ion_y

def calc_time_tau_pump(wavelength):
    OptCycPump = wavelength / AU.nm / AU.speed_of_light
    tau_pump = np.pi * 1 * OptCycPump / (4 * np.arccos(2**(-1/(2*8)))) + 10
    time = np.arange(-int(tau_pump), int(tau_pump) + 1, 1.)
    return time, OptCycPump

def calc_ion_QS_NA(time, field_pump):
    ion_nonAdiabatic = 2 * np.real(IonRate(
        time, time, field_pump, c1=111.927, e1=gamma/2*e1, a1=4*gamma*a1, a2=4*gamma**2*a2, 
        b1=b1, b2=2*gamma*b1*b2, b3=b3, p1=p1, dTi=2*gamma*dTi, Ip=Ipot, e2=1/36, e2n=30, e2d=2, b4=-16/9*b3))
    ion_analytical = analytical_rate(
        field_pump, c1, e1=gamma/2*e1, a1=4*gamma*a1, a2=4*gamma**2*a2, 
        b1=b1, b2=2*gamma*b1*b2, b3=b3, p1=p1, dTi=2*gamma*dTi, Ip=Ipot)
    return ion_nonAdiabatic, ion_analytical

def calc_fieldTotDel(field_pump, field_probe):
    return lambda x: field_pump + field_probe(x)

def calc_fields_pump_probe(time, wavelength_pump, wavelength_probe, intensity_pump, intensity_probe, OptCycPump, FWHM):
    field_pump = Electric_Field(time, wavelength_pump, intensity_pump, 0, 1*OptCycPump, pi=np.pi, N=8)
    field_probe = lambda x: Electric_Field(time + x, wavelength_probe, intensity_probe, -np.pi/2, FWHM/AtomicUnits.fs, pi=np.pi, N=8)
    return field_pump, field_probe

def calc_ion_Prob_QS_NA(time, fieldTotDel):
    an_Prob=lambda x : [1-np.exp(-np.trapz(analytical_rate(fieldTotDel(tau), c1, e1=gamma/2*e1, a1=0*4*gamma*a1, a2=4*gamma**2*a2, b1=b1, b2=2*gamma*b1*b2, b3=b3, p1=p1, dTi=2*gamma*dTi, Ip=Ipot),time)) for tau in x]
    noAdiabatic_prob=lambda x : [1-np.exp(-IonProb(time, fieldTotDel(tau),c1, e1=gamma/2*e1, a1=4*gamma*a1, a2=4*gamma**2*a2, b1=b1, b2=2*gamma*b1*b2, b3=b3, p1=p1, dTi=2*gamma*dTi, Ip=Ipot,e2=1/36, e2n=30, e2d=2,b4=-16/9*b3)) for tau in x]
    return an_Prob, noAdiabatic_prob

In [19]:
laser_file_params = [
    {
        'input_dir': '/home/user/TIPTOE/850nm/350_nm/I_1.25e+14',
        'output_ending': '850nm_350nm_1.25e+14'
    },
    {
        'input_dir': '/home/user/TIPTOE/850nm/350_nm/I_7.50e+13',
        'output_ending': '850nm_350nm_7.5e+13'
    },
    {
        'input_dir': '/home/user/TIPTOE/900nm/320_nm/I_5.00e+14',
        'output_ending': '900nm_320_nm_5e+14'
    },
    {
        'input_dir': '/home/user/TIPTOE/1200nm/320_nm/I_1.00e+14',
        'output_ending': '1200nm_320_nm_1e+14'
    },
    {
        'input_dir': '/home/user/TIPTOE/1200nm/320_nm/I_5.00e+14',
        'output_ending': '1200nm_320_nm_5e+14'
    }
]

base_dir = '.'
output_dir = '/home/user/TIPTOE/process_all_files_output'
for params in laser_file_params:
    input_dir = params['input_dir']
    output_ending = params['output_ending']
    output_interpol = f'{output_dir}/output_interpol_{output_ending}.csv'
    output_ion_prob = f'{output_dir}/ionProb_{output_ending}.csv'
    output_extract_field = f'{output_dir}/output_extract_field_0_{output_ending}.csv'
    process_laser_files(base_dir, input_dir, output_interpol, output_ion_prob, output_extract_field)

In [40]:

%load_ext autoreload
%autoreload
# file_params = [
#     ("ionProb_850nm_350nm_1.25e+14.csv", 850, 350, 1.25e14, 1e10, 0.93),
#     # ("ionProb_850nm_350nm_7.5e+13.csv", 850, 350, 7.5e13, 6e9, 0.93),
#     # ("ionProb_900nm_320_nm_5e+14.csv", 900, 320, 5e14, 4e10, 0.75),
#     # ("ionProb_1200nm_320_nm_1e+14.csv", 1200, 320, 1e14, 4e10, 0.75),
#     # ("ionProb_1200nm_320_nm_5e+14.csv", 1200, 320, 5e14, 4e10, 0.75)
# ]

params={'Multiplier': 0.3537672860419956, 'Ip': 0.5, 'αPol': 4.51, 'gamma': 2.2225332077079094, 'e0': 3.7538886827392335, 'a0': 1, 'a1': 1.0, 'b0': 1.0, 'b1': 1, 'b2': 1, 'c2': 2.445215663819714, 'p1': 3.5, 'd1': 0.22496851712539684}



gamma=params['gamma']

c2=-16/9*params['b2']*params['c2']

Multiplier,  e0, a0, a1, b0, b1, b2, p1, d1, Ip, αPol= params['Multiplier'], gamma/2*params['e0'], 4*gamma*params['a0'], params['a1'], 4*gamma**2*params['b0'], 2*gamma*params['b1']*params['a1'], params['b2'], params['p1'], params['d1'], params['Ip'], params['αPol']

params={'Multiplier': Multiplier, 'E_g': Ip, 'αPol': αPol, 'e0': e0, 'a0': a0, 'a1': a1, 'b0': b0, 'b1': b1, 'b2': b2, 'p1': p1, 'd1': d1, 'c2': c2}

params_qs = {'Multiplier': Multiplier, 'E_g': Ip, 'αPol': αPol, 'e0': e0, 'a0': a0, 'a1': a1, 'b0': b0, 'b1': b1, 'b2': b2, 'p1': p1, 'd1': d1}


laser_pulses = LaserField(cache_results=True)

# for file_name, wavelength_pump, wavelength_probe, intensity_pump, intensity_probe, FWHM in file_params:
#     file_path = f"/home/user/TIPTOE/process_all_files_output/{file_name}"
#     delay, ion_y = read_ion_Prob_data(file_path)
#     time, OptCycPump = calc_time_tau_pump(wavelength_pump)
#     field_pump, field_probe = calc_fields_pump_probe(time, wavelength_pump, wavelength_probe, intensity_pump, intensity_probe, OptCycPump, FWHM)
#     ion_nonAdiabatic, ion_analytical = calc_ion_QS_NA(time, field_pump)
#     output_file = f"/home/user/TIPTOE/plot_ion_tau_calc_output_data/ion_rate_QS_NA_{wavelength_pump}nm_{wavelength_probe}nm_{intensity_pump:.2e}.csv"
#     write_csv_rate(output_file, time, field_pump, field_probe(0), ion_nonAdiabatic, ion_analytical)




# delay, ion_y = read_ion_Prob_data("/home/user/TIPTOE/process_all_files_output/ionProb_850nm_350nm_1.25e+14.csv")
delay, ion_y = read_ion_Prob_data("/home/user/TIPTOE/process_all_files_output/ionProb_900nm_320_nm_5e+14.csv")


# ion_qs = []
# ion_na = []
# for tau in delay:
#     laser_pulses.add_pulse(850, 1.25e14, 0, 850/ AtomicUnits.nm / AtomicUnits.speed_of_light)
#     laser_pulses.add_pulse(350, 1e10, -np.pi/2, 0.93/AtomicUnits.fs, t0=-tau)
#     t_min, t_max = laser_pulses.get_time_interval()
#     time=np.arange(t_min, t_max+1, 1)
#     ion_qs.append(1-np.exp(-np.trapz(analyticalRate(time, laser_pulses, **params_qs), time)))
#     ion_na.append(1-np.exp(-IonProb(laser_pulses, **params, dt=2, dT=0.25)))
#     ion_na_rate = IonRate(time, laser_pulses, **params, dT=0.25)
#     laser_pulses.reset()

ion_qs = []
ion_na = []
for tau in delay:
    laser_pulses.add_pulse(900, 5e14, 0, 900/ AtomicUnits.nm / AtomicUnits.speed_of_light)
    laser_pulses.add_pulse(320, 4e10, -np.pi/2, 0.75/AtomicUnits.fs, t0=-tau)
    t_min, t_max = laser_pulses.get_time_interval()
    time=np.arange(t_min, t_max+1, 1)
    ion_qs.append(1-np.exp(-np.trapz(analyticalRate(time, laser_pulses, **params_qs), time)))
    ion_na.append(1-np.exp(-IonProb(laser_pulses, **params, dt=2, dT=0.25)))
    ion_na_rate = IonRate(time, laser_pulses, **params, dT=0.25)
    laser_pulses.reset()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/user/TIPTOE/kernel.py:289: RuntimeWarning:

divide by zero encountered in divide

/home/user/TIPTOE/kernel.py:290: RuntimeWarning:

divide by zero encountered in divide

/home/user/TIPTOE/kernel.py:289: RuntimeWarning:

invalid value encountered in multiply

/home/user/TIPTOE/kernel.py:292: RuntimeWarning:

divide by zero encountered in divide



In [27]:
#output_file = "/home/user/TIPTOE/plot_ion_tau_calc_output_data/ion_prob_new.csv"
#write_csv_prob(output_file, delay, ion_y, ion_qs, ion_na)



fig = go.Figure()
fig.add_trace(go.Scatter(x=delay, y=ion_y-ion_y[0], mode='lines', name='ion_y'))
fig.add_trace(go.Scatter(x=delay, y=ion_na-ion_na[0], mode='lines', name='ion_na'))
fig.add_trace(go.Scatter(x=delay, y=ion_qs-ion_qs[0], mode='lines', name='ion_qs'))
fig.show()

In [39]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=delay, y=ion_y, mode='lines+markers', name='ion_y'))
fig.add_trace(go.Scatter(x=delay, y=ion_na, mode='lines', name='ion_na'))
fig.add_trace(go.Scatter(x=delay, y=ion_qs, mode='lines', name='ion_qs'))
fig.show()

In [8]:

for file_name, wavelength_pump, wavelength_probe, intensity_pump, intensity_probe, FWHM in file_params:
    file_path = f"/home/user/TIPTOE/process_all_files_output/{file_name}"
    delay, ion_y = read_ion_Prob_data(file_path)
    time, OptCycPump = calc_time_tau_pump(wavelength_pump)
    field_pump, field_probe = calc_fields_pump_probe(time, wavelength_pump, wavelength_probe, intensity_pump, intensity_probe, OptCycPump, FWHM)
    fieldTotDel = calc_fieldTotDel(field_pump, field_probe)
    ion_qs, ion_na = calc_ion_Prob_QS_NA(time, fieldTotDel)
    ion_QS=ion_qs(delay)
    ion_NA=ion_na(delay)
    output_file = f"/home/user/TIPTOE/plot_ion_tau_calc_output_data/ion_prob_QS_NA_{wavelength_pump}nm_{wavelength_probe}nm_{intensity_pump:.2e}.csv"
    write_csv_prob(output_file, delay, ion_y, ion_QS, ion_NA)